# Delta-RS Python Example

This notebook shows how to write write and read Delta Lake tables without the need for Spark by leveraging the Delta-RS Python bindings. For more information about the delta-rs python bindings, seehttps://delta-io.github.io/delta-rs/python/

Click the Table of Contents button in the left JupyterLab sidebar (the button on the far left of this browser window that looks like a bulleted list) to see the types of examples provided. **Make sure to run all the cells above a given section, since most examples in this notebook depend on those above them**

In [1]:
import pyarrow.fs as fs
from deltalake import DeltaTable

## Read a Delta Table

In [2]:
table_uri = "s3://test/appl_stock_delta_table"
s3_options = {
    "access_key_id": "jupyteraccesskey", "secret_access_key":"jupytersupersecretkey", "aws_endpoint_url":"http://minio:9000", "AWS_STORAGE_ALLOW_HTTP":"true", "region":"us-east-1"
}
dt = DeltaTable(table_uri, storage_options=s3_options)

In [3]:
dt.version()
dt.files()

['Year=2010/Month=6/part-00000-b9ea0587-256b-4917-aaea-d46e69d5b582.c000.snappy.parquet',
 'Year=2014/Month=4/part-00000-5bf2f4a7-352d-4f16-9a9d-75aaa8579b10.c000.snappy.parquet',
 'Year=2015/Month=3/part-00000-0385f515-0f3e-4166-8d0f-0ea4dbc0ff32.c000.snappy.parquet',
 'Year=2010/Month=5/part-00000-f448aef9-94dc-4b0d-a1e1-0af9a4ccb56c.c000.snappy.parquet',
 'Year=2011/Month=8/part-00000-6e545315-fe2b-4ea4-a595-768795d63de5.c000.snappy.parquet',
 'Year=2011/Month=12/part-00000-e1741bac-d03e-4114-9d0a-cfa07e042f40.c000.snappy.parquet',
 'Year=2013/Month=4/part-00000-53be96b1-8035-4e01-9177-324d47e082f3.c000.snappy.parquet',
 'Year=2014/Month=3/part-00000-0e4d4ea5-790d-45b1-a1c2-50c96d7d66ee.c000.snappy.parquet',
 'Year=2013/Month=6/part-00000-c300dc0b-badc-483f-9644-22f342c8fa61.c000.snappy.parquet',
 'Year=2014/Month=6/part-00000-f4931d63-751c-4db5-8f14-ea63f99de18c.c000.snappy.parquet',
 'Year=2011/Month=10/part-00000-08278325-101d-43f0-a7bc-77a0865cdcd5.c000.snappy.parquet',
 'Year=2